In [ ]:
!pip install sentencepiece
!pip install skorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 kB 6.2 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from skorch import NeuralNetClassifier
from transformers import XLNetTokenizer, XLNetForSequenceClassification
from sklearn.metrics import classification_report, accuracy_score
import joblib

from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

import joblib

# Load the pickled file
oversample_train_new = joblib.load('/content/drive/MyDrive/oversample_trainNEW.pkl')

# Separate features and labels
X = oversample_train_new['tweet']
y = oversample_train_new['label']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Load XLNet tokenizer, encode data sets
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
encoded_train = tokenizer(X_train.tolist(), padding=True, truncation=True, return_tensors='pt')
encoded_test = tokenizer(X_test.tolist(), padding=True, truncation=True, return_tensors='pt')

# Convert labels to tensor
y_train_tensor = torch.tensor(y_train.values)
y_test_tensor = torch.tensor(y_test.values)

# Define the XLNet model with a classification layer
class XLNetClassifier(nn.Module):
    def __init__(self, num_labels=3):
        super(XLNetClassifier, self).__init__()
        self.xlnet = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=num_labels)
        print(self.xlnet)

    def forward(self, input_ids, attention_mask, token_type_ids=None):
        outputs = self.xlnet(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.logits

    def get_loss(self, y_pred, y_true, X=None, training=False):
        y_true = torch.tensor(y_true, dtype=torch.int64, device=y_pred.device)
        loss = nn.CrossEntropyLoss()(y_pred, y_true)
        return loss

from skorch.callbacks import EpochScoring

# Define a custom callback to calculate validation loss
def validation_loss(net, X, y):
    y_pred = net.predict(X)
    return nn.CrossEntropyLoss()(torch.tensor(y_pred), torch.tensor(y))

# Define the neural network classifier using Skorch
net = NeuralNetClassifier(
    XLNetClassifier,
    module__num_labels=2,
    optimizer=optim.Adam,
    max_epochs=3,
    lr=5e-5,
    batch_size=8,
    iterator_train__shuffle=True,
    callbacks=[
        ('val_loss', EpochScoring(validation_loss, lower_is_better=True)),  # Calculate validation loss
        ('val_acc', EpochScoring('accuracy', lower_is_better=False))  # Calculate validation accuracy
    ]
)

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
# Train the model
net.fit(encoded_train, y_train_tensor, epochs=3)

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0-11): 12 x XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (sequence_summary): SequenceSummary(
    (summary): Linear(in_features=768, out_features=768, bias=True)
    (activation): Tanh()
    (first_dropout): Identity()
    (last

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=XLNetClassifier(
    (xlnet): XLNetForSequenceClassification(
      (transformer): XLNetModel(
        (word_embedding): Embedding(32000, 768)
        (layer): ModuleList(
          (0-11): 12 x XLNetLayer(
            (rel_attn): XLNetRelativeAttention(
              (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (ff): XLNetFeedForward(
              (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (layer_1): Linear(in_features=768, out_features=3072, bias=True)
              (layer_2): Linear(in_features=3072, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
              (activation_function): GELUActivation()
            )
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (dropout): Dropout(p=0.1, inplace=False)

In [ ]:
y_pred = net.predict(encoded_test)

# model evaluation
print("Accuracy:", accuracy_score(y_test_tensor, y_pred))
print("\nClassification report:\n", classification_report(y_test_tensor, y_pred))

Accuracy: 0.4946164199192463

Classification report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00      6008
           1       0.49      1.00      0.66      5880

    accuracy                           0.49     11888
   macro avg       0.25      0.50      0.33     11888
weighted avg       0.24      0.49      0.33     11888



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
